# Groth16 Python notebook

Groth16 implementation according to the Rareskills ZK book:

https://www.rareskills.io/zk-book


## 1. Create QAP from R1CS


In [ ]:
!pip install galois py_ecc

In [22]:
# Imports & initialize finite field. This takes a while to run

import numpy as np
import galois
from py_ecc.bn128 import G1, G2, FQ12, curve_order, add, multiply, pairing, Z1, neg, final_exponentiate
import random
from functools import reduce

GF = galois.GF(curve_order)

In [23]:
# Define the R1CS for
# 5x^3 - 4y^2x^2 + 13xy^2 + x^2 - 10y

L = GF(np.array([
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0],
  [0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0, 0, 0, curve_order-1, 0],
  [0, 0, 0, 0, 0, 0, 1, curve_order-1, 1, 0, 1],
]))

R = GF(np.array([
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]))

O = GF(np.array([
  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], # a = x * x
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], # b = y * y
  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], # c = 5a * x
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], # d = 4b * a
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], # e = 13x * b
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], # f = 10 * y
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], # g = a - f
  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], # out = c - d + e + g
]))

# Calculate witness

x = GF(23)
y = GF(5)

a = x * x
b = y * y
c = 5 * a * x
d = 4 * b * a
e = 13 * x * b
f = 10 * y
g = a - f
out = c - d + e + g

w = GF([1, out, x, y, a, b, c, d, e, f, g])

assert all(np.equal(np.matmul(L, w) * np.matmul(R, w), np.matmul(O, w))), "not equal"


In [24]:
# Interpolate the polynomials

def interpolate_column(col):
    xs = GF(np.array([1, 2, 3, 4, 5, 6, 7, 8]))
    return galois.lagrange_poly(xs, col)

U = [interpolate_column(L[:, i]) for i in range(L.shape[1])]
V = [interpolate_column(R[:, i]) for i in range(R.shape[1])]
W = [interpolate_column(O[:, i]) for i in range(O.shape[1])]

In [ ]:
# Inspect the polynomials

assert(U[2](5) == 13)

print(len(U))

print(U[6].degree)

## 2. QAP Evaluation
First we evaluate the QAP without encryption. If the following holds:

U * V == W + h * t

Then we have a satisfied R1CS.

In [6]:
# Compute the inner product of each polynomial and the witness vector

def inner_product(polys, witness):
  mul_ = lambda x, y: x * y
  sum_ = lambda x, y: x + y
  return reduce(sum_, map(mul_, polys, witness))

Ua = inner_product(U, w)
Va = inner_product(V, w)
Wa = inner_product(W, w)

# Calcuate h
# # t = (x - 1)(x - 2) ... (x - 8)

t = galois.Poly([1], field=GF)

for i in range(1, 8):
    factor = galois.Poly([1, curve_order - i], field=GF)
    t *= factor

h = (Ua * Va - Wa) // t

assert Ua * Va == Wa + h * t

## 3.  Groth16

This code assumes that the prover has already computed Ua, Va, Wa and h as shown above.

In [17]:
def encrypted_poly_eval(ec_points, coefficients):
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(ec_points, coefficients)), Z1)

# -- TRUSTED SETUP PHASE --
# Secret parameters for polynomial evaluation and forgery prevention
# Everything that's not an elliptic curve point is TOXIC WASTE!
# Note: These parameters should be randomly generated in a real-world scenario.
tau_secret = GF(1337)  # Secret point for polynomial evaluation
alpha_secret = GF(17)
beta_secret = GF(32)
gamma_secret = GF(99)
delta_secret = GF(131)

# Generation of Elliptic Curve Points
alpha_g1_point = multiply(G1, int(alpha_secret))
beta_g1_point = multiply(G1, int(beta_secret))
beta_g2_point = multiply(G2, int(beta_secret))
gamma_g2_point = multiply(G2, int(gamma_secret))
delta_g1_point = multiply(G1, int(delta_secret))
delta_g2_point = multiply(G2, int(delta_secret))

# Calculation of τ powers for encrypted polynomial evaluation
polynomial_degree = Ua.degree
tau_powers_g1 = [multiply(G1, int(tau_secret ** i)) for i in range(polynomial_degree + 1)]
tau_powers_g2 = [multiply(G2, int(tau_secret ** i)) for i in range(polynomial_degree + 1)]

# Precomputation for prover's C calculation
# Combines evaluations of polynomials U, V, W at τ with α and β
beta_U_tau = [beta_secret * poly(tau_secret) for poly in U]
alpha_V_tau = [alpha_secret * poly(tau_secret) for poly in V]
tau_W = [poly(tau_secret) for poly in W]
combined_C_tau = [w + beta_u + alpha_v for w, beta_u, alpha_v in zip(tau_W, beta_U_tau, alpha_V_tau)]

# Number of public inputs
num_public_inputs = 2

# Splitting powers of τ for C into public and private parts
# and convert into elliptic curve points
public_tau_powers_C = [multiply(G1, int(c / gamma_secret)) for c in combined_C_tau[:num_public_inputs]]
private_tau_powers_C = [multiply(G1, int(c / delta_secret)) for c in combined_C_tau[num_public_inputs:]]

# Powers of τ for h(τ)t(τ)
# Adjusted by dividing with delta for verification multiplication
tau_powers_HT_G1 = [multiply(G1, int(tau_secret ** i * t(tau_secret) / delta_secret)) for i in range(polynomial_degree + 1)]

# -- PROVER PHASE --
# Prover generates random variables for proof construction
# Ua, Va, Wa and h have been previously computed from a satisfied QAP.
random_r, random_s = GF(177), GF(139)

# Prover computes encrypted polynomial evaluations A1, B1, B2
A1 = encrypted_poly_eval(tau_powers_g1, Ua.coefficients()[::-1])
B1 = encrypted_poly_eval(tau_powers_g1, Va.coefficients()[::-1])
B2 = encrypted_poly_eval(tau_powers_g2, Va.coefficients()[::-1])

# Final adjustments with random variables and elliptic curve points
A1_final = add(add(A1, alpha_g1_point), multiply(delta_g1_point, int(random_r)))
B1_final = add(add(B1, beta_g1_point), multiply(delta_g1_point, int(random_s)))
B2_final = add(add(B2, beta_g2_point), multiply(delta_g2_point, int(random_s)))

# Prover computes encrypted polynomial evaluation for private inputs
HT_G1 = encrypted_poly_eval(tau_powers_HT_G1, h.coefficients()[::-1])
private_input_C = encrypted_poly_eval(private_tau_powers_C, w[num_public_inputs:])
scaled_A1 = multiply(A1_final, int(random_s))
scaled_B1 = multiply(B1_final, int(random_r))
product_delta = multiply(delta_g1_point, int(random_r * random_s))
C1_final = add(add(add(add(private_input_C, HT_G1), scaled_A1), scaled_B1), neg(product_delta))

# -- VERIFIER PHASE --
# Verifier computes encrypted polynomial evaluation for public inputs
public_input_ec = encrypted_poly_eval(public_tau_powers_C, w[:num_public_inputs])

# Verifier checks the pairing equations
pairing_term1 = pairing(B2_final, neg(A1_final))
pairing_term2 = pairing(beta_g2_point, alpha_g1_point)
pairing_term3 = pairing(gamma_g2_point, public_input_ec)
pairing_term4 = pairing(delta_g2_point, C1_final)

# Final exponentiation to check the proof's validity
assert final_exponentiate(pairing_term1 * pairing_term2 * pairing_term3 * pairing_term4) == FQ12.one()